# ML4NLP1
## Starting Point for Exercise 1, part I

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part I.

One of the goals of this exercise is o make you acquainted with sklearn and related libraries like pandas and numpy. You will probably need to consult the documentation of those libraries:
- sklearn: [Documentation](https://scikit-learn.org/stable/user_guide.html)
- Pandas: [Documentation](https://pandas.pydata.org/docs/#)
- Numpy: [Documentation](https://numpy.org/doc/)

**Importing files to Google Colab:** If you have never used Colab or never uploaded a file to Colab, quickly skim over an introduction: [Introduction on medium](https://medium.com/@master_yi/importing-datasets-in-google-colab-c816fc654f97).

We're using the second method mentioned in the blogpost: (1) upload the four files `x_train.txt` and `y_train.txt`, `x_test.txt` and `y_test.txt` to a directory in Google Drive and (2) adjust the paths in the second cell to point to your uploaded files.

Then execute the first cell to give Colab permission to access the two files.

In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

In [3]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:00<00:00, 202MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:00<00:00, 157MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 114MB/s]


In [118]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [119]:
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})
# write train_df to csv with tab as separator
train_df.to_csv('train_df.csv', index=False, sep='\t')
# comibne x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [4]:
train_df.head()

,text,label
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


In [35]:
# get list of all labels
labels = train_df['label'].unique().tolist()
print(labels)

['est', 'swe', 'mai', 'oci', 'tha', 'orm', 'lim', 'guj', 'pnb', 'zea', 'krc', 'hat', 'pcd', 'tam', 'vie', 'pan', 'szl', 'ckb', 'fur', 'wuu', 'arz', 'ton', 'eus', 'map-bms', 'glk', 'nld', 'bod', 'jpn', 'arg', 'srd', 'ext', 'sin', 'kur', 'che', 'tuk', 'pag', 'tur', 'als', 'koi', 'lat', 'urd', 'tat', 'bxr', 'ind', 'kir', 'zh-yue', 'dan', 'por', 'fra', 'ori', 'nob', 'jbo', 'kok', 'amh', 'khm', 'hbs', 'slv', 'bos', 'tet', 'zho', 'kor', 'sah', 'rup', 'ast', 'wol', 'bul', 'gla', 'msa', 'crh', 'lug', 'sun', 'bre', 'mon', 'nep', 'ibo', 'cdo', 'asm', 'grn', 'hin', 'mar', 'lin', 'ile', 'lmo', 'mya', 'ilo', 'csb', 'tyv', 'gle', 'nan', 'jam', 'scn', 'be-tarask', 'diq', 'cor', 'fao', 'mlg', 'yid', 'sme', 'spa', 'kbd', 'udm', 'isl', 'ksh', 'san', 'aze', 'nap', 'dsb', 'pam', 'cym', 'srp', 'stq', 'tel', 'swa', 'vls', 'mzn', 'bel', 'lad', 'ina', 'ava', 'lao', 'min', 'ita', 'nds-nl', 'oss', 'kab', 'pus', 'fin', 'snd', 'kaa', 'fas', 'cbk', 'cat', 'nci', 'mhr', 'roa-tara', 'frp', 'ron', 'new', 'bar', 'ltg'

In [32]:
len(labels)

235

In [ ]:
# T: Have a quick peek at the training data, looking at a couple of texts from different languages. Do you notice anything that might be challenging for the classification?

Several languages in the Latin family use the same alphabet, and the words they form are extremely similar。

In [ ]:
# T: How many instances per label are there in the training and test set? Do you think this is a balanced dataset? Do you think the train/test split is appropriate? If not, please rearrange the data in a more appropriate way.

In [120]:
# T: How many instances per label are there in the training and test set?
labels, counts = np.unique(train_df['label'], return_counts=True)
dict(zip(labels, counts))


{'ace': 500,
 'afr': 500,
 'als': 500,
 'amh': 500,
 'ang': 500,
 'ara': 500,
 'arg': 500,
 'arz': 500,
 'asm': 500,
 'ast': 500,
 'ava': 500,
 'aym': 500,
 'azb': 500,
 'aze': 500,
 'bak': 500,
 'bar': 500,
 'bcl': 500,
 'be-tarask': 500,
 'bel': 500,
 'ben': 500,
 'bho': 500,
 'bjn': 500,
 'bod': 500,
 'bos': 500,
 'bpy': 500,
 'bre': 500,
 'bul': 500,
 'bxr': 500,
 'cat': 500,
 'cbk': 500,
 'cdo': 500,
 'ceb': 500,
 'ces': 500,
 'che': 500,
 'chr': 500,
 'chv': 500,
 'ckb': 500,
 'cor': 500,
 'cos': 500,
 'crh': 500,
 'csb': 500,
 'cym': 500,
 'dan': 500,
 'deu': 500,
 'diq': 500,
 'div': 500,
 'dsb': 500,
 'dty': 500,
 'egl': 500,
 'ell': 500,
 'eng': 500,
 'epo': 500,
 'est': 500,
 'eus': 500,
 'ext': 500,
 'fao': 500,
 'fas': 500,
 'fin': 500,
 'fra': 500,
 'frp': 500,
 'fry': 500,
 'fur': 500,
 'gag': 500,
 'gla': 500,
 'gle': 500,
 'glg': 500,
 'glk': 500,
 'glv': 500,
 'grn': 500,
 'guj': 500,
 'hak': 500,
 'hat': 500,
 'hau': 500,
 'hbs': 500,
 'heb': 500,
 'hif': 500,
 'hin'

In [121]:
# T: How many instances per label are there in the training and test set?
labels, counts = np.unique(test_df['label'], return_counts=True)
dict(zip(labels, counts))


{'ace': 500,
 'afr': 500,
 'als': 500,
 'amh': 500,
 'ang': 500,
 'ara': 500,
 'arg': 500,
 'arz': 500,
 'asm': 500,
 'ast': 500,
 'ava': 500,
 'aym': 500,
 'azb': 500,
 'aze': 500,
 'bak': 500,
 'bar': 500,
 'bcl': 500,
 'be-tarask': 500,
 'bel': 500,
 'ben': 500,
 'bho': 500,
 'bjn': 500,
 'bod': 500,
 'bos': 500,
 'bpy': 500,
 'bre': 500,
 'bul': 500,
 'bxr': 500,
 'cat': 500,
 'cbk': 500,
 'cdo': 500,
 'ceb': 500,
 'ces': 500,
 'che': 500,
 'chr': 500,
 'chv': 500,
 'ckb': 500,
 'cor': 500,
 'cos': 500,
 'crh': 500,
 'csb': 500,
 'cym': 500,
 'dan': 500,
 'deu': 500,
 'diq': 500,
 'div': 500,
 'dsb': 500,
 'dty': 500,
 'egl': 500,
 'ell': 500,
 'eng': 500,
 'epo': 500,
 'est': 500,
 'eus': 500,
 'ext': 500,
 'fao': 500,
 'fas': 500,
 'fin': 500,
 'fra': 500,
 'frp': 500,
 'fry': 500,
 'fur': 500,
 'gag': 500,
 'gla': 500,
 'gle': 500,
 'glg': 500,
 'glk': 500,
 'glv': 500,
 'grn': 500,
 'guj': 500,
 'hak': 500,
 'hat': 500,
 'hau': 500,
 'hbs': 500,
 'heb': 500,
 'hif': 500,
 'hin'

1. T: How many instances per label are there in the training and test set?
   
   Answer: 500 instances per label in training set and 500 instances per label in test set. 
   
2. Do you think this is a balanced dataset? 

   Answer: To some extend, yes, each language has the same number of instances. However, training set should contain more instances than test set.

3. Do you think the train/test split is appropriate? 
   
   Answer: No. Training set should have more instances each language than test set. Like 4:1. Here it's 1:1. 

4. If not, please rearrange the data in a more appropriate way.

In [122]:
data_set = pd.concat([train_df, test_df])

In [135]:
new_train_df = pd.DataFrame()
new_test_df = pd.DataFrame()
for label in labels:
    label_instances = data_set[data_set['label'] == label]
    train_instances = label_instances.iloc[:800]
    test_instances = label_instances.iloc[800:]
    new_train_df = new_train_df.append(train_instances)
    new_test_df = new_test_df.append(test_instances)

In [136]:
labels, counts = np.unique(new_train_df['label'], return_counts=True)
len(labels), counts

(235,
 array([800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800, 80

In [137]:
labels, counts = np.unique(new_test_df['label'], return_counts=True)
len(labels), counts

(235,
 array([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 20

In [138]:
train_df = new_train_df.reset_index(drop=True)
test_df = new_test_df.reset_index(drop=True)

In [ ]:
# T: Get a subset of the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)

In [140]:
labels

array(['ace', 'afr', 'als', 'amh', 'ang', 'ara', 'arg', 'arz', 'asm',
       'ast', 'ava', 'aym', 'azb', 'aze', 'bak', 'bar', 'bcl',
       'be-tarask', 'bel', 'ben', 'bho', 'bjn', 'bod', 'bos', 'bpy',
       'bre', 'bul', 'bxr', 'cat', 'cbk', 'cdo', 'ceb', 'ces', 'che',
       'chr', 'chv', 'ckb', 'cor', 'cos', 'crh', 'csb', 'cym', 'dan',
       'deu', 'diq', 'div', 'dsb', 'dty', 'egl', 'ell', 'eng', 'epo',
       'est', 'eus', 'ext', 'fao', 'fas', 'fin', 'fra', 'frp', 'fry',
       'fur', 'gag', 'gla', 'gle', 'glg', 'glk', 'glv', 'grn', 'guj',
       'hak', 'hat', 'hau', 'hbs', 'heb', 'hif', 'hin', 'hrv', 'hsb',
       'hun', 'hye', 'ibo', 'ido', 'ile', 'ilo', 'ina', 'ind', 'isl',
       'ita', 'jam', 'jav', 'jbo', 'jpn', 'kaa', 'kab', 'kan', 'kat',
       'kaz', 'kbd', 'khm', 'kin', 'kir', 'koi', 'kok', 'kom', 'kor',
       'krc', 'ksh', 'kur', 'lad', 'lao', 'lat', 'lav', 'lez', 'lij',
       'lim', 'lin', 'lit', 'lmo', 'lrc', 'ltg', 'ltz', 'lug', 'lzh',
       'mai', 'mal', 'map-bm

In [344]:
""" 
English: eng
German: deu
Dutch: nld
Danish: dan
Swedish: swe
Norwegian: nob

Japanese: jpn

"""

subset_labels = {'eng', 'deu', 'nld', 'dan', 'swe', 'jpn', 'nob'}
other_labels = set(labels) - subset_labels
subset_labels = subset_labels | set(list(other_labels)[:20])

In [345]:
len(subset_labels)

27

In [346]:
sub_train_set = train_df[train_df['label'].isin(subset_labels)]
sub_test_set = test_df[test_df['label'].isin(subset_labels)]
len(sub_train_set), len(sub_test_set)

(21600, 5400)

In [347]:
# T: With the following code, we wanted to encode the labels, 
# however, our cat was walking on the keyboard and some of it got changed. Can you fix it?
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
le_fitted = label_encoder.fit(sub_train_set['label'])
y_train_dev, y_test = le_fitted.transform(sub_train_set['label']), le_fitted.transform(sub_test_set['label'])

## Pipeline
Create a suitable pipeline in sklearn to preprocess the data. Think about extending the feature space.
What other features could you use to determine the language? Please include additional linguistic
features to your machine learning model for this task.

In [348]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Here we use TfidfVectorizer to replace CountVectorizer and TfidfTransformer
# Due to Eli5, if we use TfidfTransformer there would be errors. 
# This is also more clear and intuitive. 
text_clf = Pipeline([
    ('TfidfVect', TfidfVectorizer(ngram_range=(1, 2),)),
    ('LR', LogisticRegression())
])

In [349]:
text_clf.fit(sub_train_set['text'], y_train_dev)

/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('TfidfVect', TfidfVectorizer()), ('LR', LogisticRegression())])

In [350]:
scores = cross_val_score(text_clf, sub_train_set['text'], y_train_dev, scoring='accuracy', cv=3)

In [351]:
scores

array([0.96125   , 0.96013889, 0.96333333])

## Hyperparameters
To find the optimal hyperparameter settings for the classifier, use sklearn’s GridSearchCV. [hint: don’t
overdo it at the beginning, since runtime might go up fast] You are supposed to experiment with the
following hyperparameters1:
a. Penalty (Regularization)
b. Solver
c. Experiment with parameters of the Vectorizer (not required, highly advised)



In [352]:
from sklearn.model_selection import GridSearchCV

In [353]:
# text_LR = Pipeline([('CV', CountVectorizer()),('Tfidf', TfidfTransformer()),('LR', LogisticRegression())])
text_LR = Pipeline([
    ('TfidfVect', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

param_grid = {'LR__penalty': ['l1', 'l2'],
               'LR__solver': ['liblinear', 'saga'],
               'TfidfVect__ngram_range': [(1, 1), (1, 2)],
               'TfidfVect__norm': ['l1', 'l2']}

gs_LR = GridSearchCV(text_LR, param_grid, cv=5, verbose=1)
gs_LR.fit(sub_train_set['text'], y_train_dev)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/nanfangwuyu/opt/anaconda3/envs/CS-NLP/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reac

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('TfidfVect', TfidfVectorizer()),
                                       ('LR', LogisticRegression())]),
             param_grid={'LR__penalty': ['l1', 'l2'],
                         'LR__solver': ['liblinear', 'saga'],
                         'TfidfVect__ngram_range': [(1, 1), (1, 2)],
                         'TfidfVect__norm': ['l1', 'l2']},
             verbose=1)

Print out the first 5 best params. 

In [354]:
LR_df = pd.DataFrame.from_dict(gs_LR.cv_results_)
LR_df.sort_values(by=["rank_test_score"]).iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LR__penalty,param_LR__solver,param_TfidfVect__ngram_range,param_TfidfVect__norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
13,3.351609,0.256262,0.280216,0.070384,l2,saga,"(1, 1)",l2,"{'LR__penalty': 'l2', 'LR__solver': 'saga', 'T...",0.964583,0.961111,0.958565,0.963426,0.965972,0.962731,0.002623,1
15,9.400893,0.510846,0.525816,0.009730,l2,saga,"(1, 2)",l2,"{'LR__penalty': 'l2', 'LR__solver': 'saga', 'T...",0.963889,0.961343,0.959259,0.963426,0.963426,0.962269,0.001745,2
11,223.140502,7.983467,1.097650,0.161085,l2,liblinear,"(1, 2)",l2,"{'LR__penalty': 'l2', 'LR__solver': 'liblinear...",0.962731,0.960648,0.958796,0.962963,0.962731,0.961574,0.001624,3
9,108.429751,109.590767,0.378309,0.145460,l2,liblinear,"(1, 1)",l2,"{'LR__penalty': 'l2', 'LR__solver': 'liblinear...",0.962500,0.960417,0.957639,0.961806,0.964583,0.961389,0.002307,4
12,2.898128,0.263738,0.243447,0.041382,l2,saga,"(1, 1)",l1,"{'LR__penalty': 'l2', 'LR__solver': 'saga', 'T...",0.937269,0.936806,0.938889,0.937500,0.931481,0.936389,0.002550,5


## Best Model and Error Analysis
Report the hyperparameter combination for your best-performing model on the test set.
What is the advantage of grid search cross-validation? Use a confusion matrix to do your error analysis and summarize your answers in your report


In [355]:
LR_df.sort_values(by=["rank_test_score"]).iloc[0]

mean_fit_time                                                            3.351609
std_fit_time                                                             0.256262
mean_score_time                                                          0.280216
std_score_time                                                           0.070384
param_LR__penalty                                                              l2
param_LR__solver                                                             saga
param_TfidfVect__ngram_range                                               (1, 1)
param_TfidfVect__norm                                                          l2
params                          {'LR__penalty': 'l2', 'LR__solver': 'saga', 'T...
split0_test_score                                                        0.964583
split1_test_score                                                        0.961111
split2_test_score                                                        0.958565
split3_test_scor

In [356]:
LR_df.sort_values(by=["rank_test_score"]).iloc[0]['params']

{'LR__penalty': 'l2',
 'LR__solver': 'saga',
 'TfidfVect__ngram_range': (1, 1),
 'TfidfVect__norm': 'l2'}

In [357]:
best_model = Pipeline([
    # ('CV', CountVectorizer(ngram_range=(1,1))),
    # ('Tfidf', TfidfTransformer(norm='l2')), change due to eli5
    ('TfidfVect', TfidfVectorizer(ngram_range=(1,1), norm='l2')),
    ('LR', LogisticRegression(penalty='l2', solver='saga'))
])

best_model.fit(sub_train_set['text'], y_train_dev)

Pipeline(steps=[('TfidfVect', TfidfVectorizer()),
                ('LR', LogisticRegression(solver='saga'))])

In [358]:
from sklearn.metrics import confusion_matrix

y_pred = best_model.predict(sub_test_set['text'])
y_pred

array([ 0,  0,  0, ..., 26, 26, 26])

Congusion matrix for all classes

In [359]:
cm = confusion_matrix(y_test, y_pred)
cm, cm.shape

(array([[185,   0,   0,   0,   0,   0,  15,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0],
        [  0, 186,   0,   0,  10,   0,   3,   0,   0,   0,   0,   1,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0],
        [  0,   0, 200,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0],
        [  0,   0,   0, 194,   3,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   1,   0,
           0],
        [  0,   6,   0,   0, 188,   0,   1,   0,   0,   0,   0,   3,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,
           0],
        [  0,   0,   0,   0,   1, 192,   0,   0,   1,   0,   0,   6,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0],
        [  3,   0,   0

### Class Metrics
We can have precision score, recall score and f1 score on each class. 

In [360]:
from sklearn.metrics import classification_report 

report = classification_report(
    y_test, y_pred, 
    target_names=le_fitted.inverse_transform([i for i in range(len(subset_labels))]),
    )

print(report)

              precision    recall  f1-score   support

         ang       0.98      0.93      0.95       200
         bar       0.95      0.93      0.94       200
         bod       1.00      1.00      1.00       200
         dan       0.99      0.97      0.98       200
         deu       0.92      0.94      0.93       200
         dsb       1.00      0.96      0.98       200
         eng       0.80      0.95      0.87       200
         hat       1.00      0.98      0.99       200
         hrv       0.99      0.98      0.99       200
         hun       1.00      0.97      0.98       200
         jav       1.00      0.97      0.98       200
         jpn       0.80      0.97      0.88       200
         kab       1.00      0.94      0.97       200
         kbd       1.00      0.99      1.00       200
         kor       1.00      0.99      0.99       200
         ksh       0.99      0.97      0.98       200
         lad       0.99      0.98      0.99       200
         mar       1.00    

### Diagonal Elements:

Watching on the top-left to the bottom-right elements, all of these diagonal elements are close to 200, the total number of samples each label in test set. This means the true positive values for each label is relatively high, indicating that the classifier performs well on most classes.

### Non-Diagonal Elements:

However, there are also non-diagonal elements, which are errors (wrong predictions). The maximum number a class is misclassified is 15, which is the number 'ang' texts are misclassified as 'eng' texts. Another big number is 14, which shows the count of 'pcd' samples are misclassified as 'eng' texts. To be noticed, we can find that there are also some misclassifications from 'eng' to 'ang' or 'pcd' languages, though a bit fewer. These results indicate that there are some confusion between these pairs of languages. 

In [376]:
cm[0][6], cm[24][6]

(15, 14)

In [377]:
le_fitted.inverse_transform([0, 6]), le_fitted.inverse_transform([24, 6])

(array(['ang', 'eng'], dtype=object), array(['pcd', 'eng'], dtype=object))

## Model Explaination
"Now that you have your best model, it’s time to dive deep into understanding how the model makes predictions. 
It is important that we can explain and visualize our models to improve task performance. Explainable models help 
characterize model fairness, transparency, and outcomes. Let's try to understand what our best-performing logistic 
regression classification model has learned. **Generate a feature importance table for the top ten features 
(please have the features named) for the languages English, Swedish, Norwegian, and Japanese. What is 
more important, extra features or the outputs of the vectorizer, discuss**."

In [378]:
# languages = ['eng', 'swe', 'nob']
languages = ['eng', 'swe', 'nob', 'jpn']

In [364]:
%pip install eli5

Note: you may need to restart the kernel to use updated packages.


In [379]:
from sklearn.feature_extraction.text import TfidfVectorizer

best_model = Pipeline([
    # ('CV', CountVectorizer(ngram_range=(1,1))),
    # ('Tfidf', TfidfTransformer(norm='l2')), change due to eli5
    ('TfidfVect', TfidfVectorizer(ngram_range=(1,1), norm='l2')),
    ('LR', LogisticRegression(penalty='l2', solver='saga'))
])

best_model.fit(sub_train_set['text'], y_train_dev)

Pipeline(steps=[('TfidfVect', TfidfVectorizer()),
                ('LR', LogisticRegression(solver='saga'))])

In [381]:
import eli5

eli5.show_weights(best_model, top = 10, target_names = le_fitted.inverse_transform([i for i in range(len(subset_labels))]), targets = languages) 

In [367]:
#

## Ablation Study
Lastly, you will condact a small ablation study. First, choose the two languages for which the classifier worked best. 
Next, re-fit the best working model several times, each time reducing the number of characters per instance in the 
training set (1. All characters, 2. 500 characters, 3. 100 characters). How does the ablation affect the 
performance of the classifier?

In [368]:
report = classification_report(
    y_test, y_pred, 
    target_names=le_fitted.inverse_transform([i for i in range(len(subset_labels))]),
    output_dict=True
    )


In [369]:
report.pop('accuracy')
report.pop('macro avg') 
report.pop('weighted avg')

{'precision': 0.971898264971494,
 'recall': 0.9690740740740741,
 'f1-score': 0.969793693483971,
 'support': 5400}

In [370]:
ranked_labels = sorted(report, key=lambda x: (report[x]['precision'], report[x]['recall'], report[x]['f1-score']), reverse=True)
print(ranked_labels)
ranked_report = {label: report[label] for label in ranked_labels}
print(ranked_report)


['bod', 'nav', 'swe', 'kbd', 'kor', 'urd', 'mkd', 'hat', 'hun', 'jav', 'mar', 'dsb', 'kab', 'hrv', 'ksh', 'nob', 'lad', 'dan', 'mwl', 'nld', 'ang', 'nds-nl', 'bar', 'deu', 'pcd', 'jpn', 'eng']
{'bod': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 200}, 'nav': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 200}, 'swe': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 200}, 'kbd': {'precision': 1.0, 'recall': 0.995, 'f1-score': 0.9974937343358395, 'support': 200}, 'kor': {'precision': 1.0, 'recall': 0.99, 'f1-score': 0.9949748743718593, 'support': 200}, 'urd': {'precision': 1.0, 'recall': 0.99, 'f1-score': 0.9949748743718593, 'support': 200}, 'mkd': {'precision': 1.0, 'recall': 0.985, 'f1-score': 0.9924433249370278, 'support': 200}, 'hat': {'precision': 1.0, 'recall': 0.98, 'f1-score': 0.98989898989899, 'support': 200}, 'hun': {'precision': 1.0, 'recall': 0.97, 'f1-score': 0.9847715736040609, 'support': 200}, 'jav': {'precision': 1.0, 'recall':

The two languages for which the classifier worked best are: 'bod', 'nav'

In [371]:
ablation_train = sub_train_set[sub_train_set['label'].isin(('bod', 'nav'))]
ablation_test = sub_test_set[sub_test_set['label'].isin(('bod', 'nav'))]
ablation_y_train = le_fitted.transform(ablation_train['label'])
ablation_y_test = le_fitted.transform(ablation_test['label'])

In [372]:
from sklearn.metrics import accuracy_score

limits = [None, 500, 100, 10]



for limit in limits:

    best_model = Pipeline([
    ('TfidfVect', TfidfVectorizer(ngram_range=(1,1), norm='l2')),
    ('LR', LogisticRegression(penalty='l2', solver='saga'))
])
    
    if limit == None:
        ablation_X_train = ablation_train['text']
    else:
        ablation_X_train = [text[:limit] for text in ablation_train['text']]
        
    
    best_model.fit(ablation_X_train, ablation_y_train)

    ablation_y_pred = best_model.predict(ablation_test['text'])

    print("Accuracy when length is {}:".format("original" if limit == None else limit), accuracy_score(ablation_y_test, ablation_y_pred))


Accuracy when length is original: 1.0
Accuracy when length is 500: 1.0
Accuracy when length is 100: 1.0
Accuracy when length is 10: 1.0


In [373]:
print(ablation_train.iloc[0])
print(ablation_train.iloc[-1])

text     ༡༩༣༥ལོར་ཞང་པོ་དང་ལྷན་དུ་ནན་ཅིང་དུ་སློབ་སྦྱོང་མ...
label                                                  bod
Name: 17600, dtype: object
text     Chąąneiłhizii éí chéłchaaʼ dah yikahjįʼ atah d...
label                                                  nav
Name: 115999, dtype: object


As a result, the ablation has no influence on the accuracy. This is probably because these two languages ( 'nav' and 'bod' ) use compeletly different vocabulary, they are very easy to distinguish by the best model. In this case, length of the text is slightly influencial. 

However, if we choose other two languages like 'nob' and 'swe', when the length reduces to 10, there is a obvious drop in accuracy. 

In [374]:
ablation_train = sub_train_set[sub_train_set['label'].isin(('nob', 'swe'))]
ablation_test = sub_test_set[sub_test_set['label'].isin(('nob', 'swe'))]
ablation_y_train = le_fitted.transform(ablation_train['label'])
ablation_y_test = le_fitted.transform(ablation_test['label'])

In [375]:
from sklearn.metrics import accuracy_score

limits = [None, 500, 100, 10]



for limit in limits:

    best_model = Pipeline([
    ('TfidfVect', TfidfVectorizer(ngram_range=(1,1), norm='l2')),
    ('LR', LogisticRegression(penalty='l2', solver='saga'))
])
    
    if limit == None:
        ablation_X_train = ablation_train['text']
    else:
        ablation_X_train = [text[:limit] for text in ablation_train['text']]
        
    
    best_model.fit(ablation_X_train, ablation_y_train)

    ablation_y_pred = best_model.predict(ablation_test['text'])

    print("Accuracy when length is {}:".format("original" if limit == None else limit), accuracy_score(ablation_y_test, ablation_y_pred))


Accuracy when length is original: 1.0
Accuracy when length is 500: 1.0
Accuracy when length is 100: 1.0
Accuracy when length is 10: 0.9225
